<a href="https://colab.research.google.com/github/alexk2206/tds_capstone/blob/Domi-DEV/Productive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Productive Notebook

In [11]:
!pip install evaluate
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatib

In [30]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import urllib
from itertools import chain, combinations
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, pipeline, Trainer, DataCollatorWithPadding
import torch
import requests
import evaluate
import numpy as np
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union



### Preprocess dataset

Here we split the QA-dataset into train and validation dataset.
Additionnaly, we prepare the dataset to later be useful for response-generation and fine-tuning of a model

❎ Please insert code: load dataset into variable q ❎

In [10]:
# Example dataset

url = "https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/datasets/sampled_qa_dataset_easy.json"
data = pd.read_json(url)
# Convert to DataFrame for easy handling
df = pd.DataFrame(data)

# Map the intended answer to the index of the option
df['label'] = df.apply(lambda x: np.array([1 if option in x['intended_answer'] else 0 for option in x['options']]) if x['type'] in ['SINGLE_SELECT', 'MULTI_SELECT'] else np.array([0]), axis=1)
df['stratify_key'] = df['difficulty'] + '_' + df['type']

# Stratified Train-Validation Split
train_df, val_df = train_test_split(
    df,
    train_size=0.8,
    stratify=df['stratify_key'],
    random_state=42
)

In [9]:
print(df)

                                             question           type  \
0                         What type of company is it?  SINGLE_SELECT   
1   Would you like to receive marketing informatio...  SINGLE_SELECT   
2                   What is the size of your company?  SINGLE_SELECT   
3                                          Next steps  SINGLE_SELECT   
4                   What kind of follow up is planned   MULTI_SELECT   
..                                                ...            ...   
95                  What is the size of your company?  SINGLE_SELECT   
96                            Data processing consent  SINGLE_SELECT   
97                        What type of company is it?  SINGLE_SELECT   
98                           Who to copy in follow up   MULTI_SELECT   
99                            Data processing consent  SINGLE_SELECT   

                                              options  \
0   [Construction company, Craft enterprises, Scaf...   
1                    

### Generate model output

After the creation of the QA-dataset, it's time for generating model output for different Huggingface models.

In [144]:
def model_output(model, tokenizer, questions):
    '''
    model_output -> creates output for every question in the dataset and safes it in a list of dicts. One dic has keys 'answer', 'predicted_answer', 'type'
    parameters:
    - model: one hugging face model
    - tokenizer: hugging face tokenizer
    - questions: QA-dataset in json format
    '''
    answer_comparison = []
    mc_answer_comparison = []

    for index, question in questions.iterrows():
        context = question['context']
        question_text = question['question']
        options = question['options']
        question_type = question['type']
        difficulty = question['difficulty']

        if question_type == "MULTI_SELECT":
          intended_answer, intended_answer_binary, predicted_answer_binary, predicted_answer = multi_select_model_output(model, tokenizer, question)
        if question_type == "SINGLE_SELECT":
          intended_answer, intended_answer_binary, predicted_answer_binary, predicted_answer_words = single_select_model_output(model, tokenizer, question)
        if question_type == "TEXT":
          predicted_answer = text_model_output(question)
        if question_type == "NUMBER":
          predicted_answer = number_model_output(model, tokenizer, question)
        if question_type == "DATE":
          predicted_answer = date_model_output(model, tokenizer, question)
        else:
          continue

        print(f"Question: {question_text}")
        print(f"Context: {context}")
        print(f"Answer: {intended_answer}")
        print(f"Predicted Answer: {predicted_answer}\n")
        if question_type in ["MULTI_SELECT", "SINGLE_SELECT"]:
          mc_answer_comparison.append({'intended_answer_binary': intended_answer_binary, 'predicted_answer_binary': predicted_answer_binary, 'intended_answer': intended_answer, 'predicted_answer': predicted_answer, 'type': question_type, 'difficulty': difficulty})
        else:
          answer_comparison.append({'intended_answer': intended_answer, 'predicted_answer': predicted_answer, 'type': question_type, 'difficulty': difficulty})
    return mc_answer_comparison, answer_comparison


Single-select output

In [156]:
def single_select_model_output(model, tokenizer, question):
    '''
    Handles a question, its context and its options for a single-select question and generates output
    parameters:
    - model: one hugging face model
    - tokenizer: hugging face tokenizer
    - question: one question of the QA-dataset as a dictionary
    output:
    - answer: the correct/intended answer as a list of a string
    - predicted_answer: the predicted answer as a list of a string
    '''
    intended_answer = question['intended_answer']
    options = question['options']

    # creating input ids by tokenizing the question
    input_ids = tokenize_function(question, tokenizer)
    input_ids = {key: torch.tensor(array) for key, array in input_ids.items()}

    # generating the output
    outputs = model(**input_ids)
    logits = outputs.logits  # Shape: [batch_size, num_choices]

    # Predict the option with the highest score
    predicted_option = torch.argmax(logits, dim=1).item()

    predicted_binary = [0] * len(options)
    predicted_binary[predicted_option] = 1

    intended_binary = [1 if option == intended_answer else 0 for option in options]

    return intended_answer, intended_answer_binary, predicted_answer_binary, options[predicted_option]


Multi-select output

In [157]:
def multi_select_model_output(model, tokenizer, question):
    '''
    Handles a question, its context and its options for a multi-select question and generates output as a list of indices of the predicted answers. Ticks every option whose probability is at least 90% of the best option (softmax)
    parameters:
    - model: one hugging face model
    - tokenizer: hugging face tokenizer
    - question: one question of the QA-dataset as a dictionary
    output:
    - answer: the correct/intended answers as a list of strings
    - predicted_answer: the predicted answers as a list of strings
    '''
    intended_answer = question['intended_answer']
    options = question['options']

    # creating input ids by tokenizing the question
    input_ids = tokenize_function(question, tokenizer)
    input_ids = {key: torch.tensor(array) for key, array in input_ids.items()}

    # generating the output
    outputs = model(**input_ids)
    logits = outputs.logits  # Shape: [batch_size, num_choices]

    # Find all indices to have at least 70% of the max score
    max_score = logits.max().item()
    threshold = 0.7 * max_score
    high_score_options = (logits >= threshold).nonzero(as_tuple=True)[1]  # Get the indices of valid options

    # List the corresponding options
    high_score_answers = [options[idx] for idx in high_score_options.tolist()]
    intended_answer_binary = [1 if option == intended_answer else 0 for option in options]

    predicted_answer_binary = [1 if option in high_score_answers else 0 for option in options]
    print(intended_answer)
    print(high_score_answers)

    return intended_answer, intended_answer_binary, predicted_answer_binary, high_score_answers



Text output

In [ ]:
def text_model_output(question):
    '''
    Handles an open text question and summarizes it
    parameter:
    - question: one question of the QA-dataset as a dictionary
    output:
    - answer: the full context of the question as a string
    - summary: the generated summary as a string
    '''
    intended_answer = question['context']
    summarization_pipeline = pipeline("text-summarization")
    summary = summarization_pipeline(intended_answer, max_length=100, min_length=30, do_sample=False)
    return intended_answer, summary[0]['summary_text']



Phone Number output

In [ ]:
def number_model_output(model, tokenizer, question):
    '''
    Handles a question where the context should contain a phone number and generates an answer to that question
    '''
    intended_answer = question['intended_answer']

    input_ids = tokenize_function(question, tokenizer)
    output = model(**input_ids)
    predicted_number = output.logits.item()

    return intended_answer, predicted_number



Date output

In [ ]:
def date_model_output(model, tokenizer, question):
    '''
    Handles a question where the context should contain a date and generates an answer to that question
    '''
    intended_answer = question['intended_answer']

    input_ids = tokenize_function(question, tokenizer)
    output = model(**input_ids)
    predicted_date = output.logits.item()

    return intended_answer, predicted_date



Accuracy

In [ ]:
def accuracy(answer_comparison):
    '''
    Computes the total accuracy and accuracy for each question type for the passed list of dicts. One dict in the list is one question with keys 'answer', 'predicted_answer', 'type'
    parameters:
    - list of dicts with entries 1) predicted answer 2) answer 3) type of question
    '''
    correct_multi_select = 0
    correct_single_select = 0
    correct_text = 0
    correct_number = 0
    correct_date = 0
    correct_total = 0
    total = 0

    for entry in answer_comparison:
        question_type = entry['type']
        if entry['intended_answer'] == entry['predicted_answer']:
            if question_type == 'MULTI_SELECT':
                correct_multi_select += 1
                total_multi_select += 1
            elif question_type == 'SINGLE_SELECT':
                correct_single_select += 1
                total_single_select += 1
            elif question_type == 'TEXT':
                correct_text += 1
                total_text += 1
            elif question_type == 'NUMBER':
                correct_number += 1
                total_number += 1
            elif question_type == 'DATE':
                correct_date += 1
                total_date += 1
            else:
              continue
            correct_total += 1
        total += 1
    accuracy_total = correct_total / total
    accuracy_multi_select = correct_multi_select / total_multi_select
    accuracy_single_select = correct_single_select / total_single_select
    accuracy_text = correct_text / total_text
    accuracy_number = correct_number / total_number
    accuracy_date = correct_date / total_date
    return accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date
'''
print_out_model_quality: takes the computations of function accuracy() and prints them out
parameters:
- accuracy_total
- accuracy_multi_select
- accuracy_single_select
- accuracy_text
- accuracy_number
- accuracy_date
'''
def print_out_model_quality(accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date):
    # accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date = accuracy(model, tokenizer, questions)
    print(f"""Accuracy values of model: {model.name_or_path}\n
    Total: {accuracy_total}\n
    Multi-select: {accuracy_multi_select}\n
    Single-select: {accuracy_single_select}\n
    Text: {accuracy_text}\n
    Number: {accuracy_number}\n
    Date: {accuracy_date}\n""")
    return accuracy_total, accuracy_multi_select, accuracy_single_select, accuracy_text, accuracy_number, accuracy_date




### Fine-tuning a model


In [29]:

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [99]:
def fine_tune_model(train_dataset, val_dataset, tokenizer, model):
    # Define training arguments
    training_args = TrainingArguments("trainer",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_steps=10,
        load_best_model_at_end=True
    )
    data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)

    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

    # Define Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

def compute_metrics(eval_preds, pretrained_dataset_name):
    metric = evaluate.load("glue", pretrained_dataset_name)
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_function(example, tokenizer):
    '''
    Converts the string input, which is a question with its context and the given options for multi-/single-select questions, into IDs the model later can make sense of. Distinguishes between multi-/single-select and the other questions
    parameters:
    - expample: question of the QA-dataset with all its entries (question, context, options, type are urgently necessary)
    - tokenizer: tokenizer of the model
    output:
    - tokenized: tokenized input example
    '''
    if example["type"] == "SINGLE_SELECT" or example["type"] == "MULTI_SELECT":
      number_of_options = len(example["options"])
      first_sentence = [[example["context"]] * number_of_options]  # Repeat context for each option
      second_sentence = [[example["question"] + " " + option] for option in example["options"]]  # Pair with each option
      tokenized = tokenizer(
          sum(first_sentence, []),
          sum(second_sentence, []),
          padding="longest",
          truncation=True
      )
      # Un-flatten
      return {k: [v[i:i+number_of_options] for i in range(0, len(v), number_of_options)] for k, v in tokenized.items()}

    else:
      tokenized = tokenizer(
          example["context"],
          example["question"],
          truncation=True,
          max_length=512,
          padding="max_length",
          return_tensors="pt"
      )
      tokenized["labels"] = 0
    return tokenized

In [55]:
model = AutoModelForMultipleChoice.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")


In [42]:
tryout = train_df[train_df['type'] == 'MULTI_SELECT']
tryout = tryout.tail(1)

In [100]:
tryout

,question,type,options,intended_answer,context,difficulty,label,stratify_key
76,Searches a solution for,MULTI_SELECT,"[Scan business cards, Clean up CRM, Extract da...","[Extract data from emails, Improve CRM data qu...",I'm trying to find a way to extract data from ...,easy,"[0, 0, 1, 1, 1]",easy_MULTI_SELECT


In [158]:
for index, question in tryout.iterrows():
  tokenized = tokenize_function(question, tokenizer)
  question_text = question['question']
  options = question['options']
  intended_answer = question['intended_answer']
  context = question['context']

In [109]:
[tokenizer.decode(tokenized["input_ids"][0][i]) for i in range(len(question["options"]))]

["[CLS] I ' m trying to find a way to extract data from emails, improve the quality of my CRM data, and capture contacts I collect at trade fairs. [SEP] Searches a solution for Scan business cards [SEP] [PAD] [PAD] [PAD]",
 "[CLS] I ' m trying to find a way to extract data from emails, improve the quality of my CRM data, and capture contacts I collect at trade fairs. [SEP] Searches a solution for Clean up CRM [SEP] [PAD] [PAD] [PAD]",
 "[CLS] I ' m trying to find a way to extract data from emails, improve the quality of my CRM data, and capture contacts I collect at trade fairs. [SEP] Searches a solution for Extract data from emails [SEP] [PAD] [PAD]",
 "[CLS] I ' m trying to find a way to extract data from emails, improve the quality of my CRM data, and capture contacts I collect at trade fairs. [SEP] Searches a solution for Improve CRM data quality [SEP]",
 "[CLS] I ' m trying to find a way to extract data from emails, improve the quality of my CRM data, and capture contacts I collec

In [110]:
tokenized_1 = {key: torch.tensor(array) for key, array in tokenized.items()}
# Pass to the model
outputs = model(**tokenized_1).logits

In [112]:
# print(outputs.shape)
answer = torch.nn.functional.softmax(outputs, dim=-1)[0].argmax().item()

print('======= Test Case 1 =======')
print(f"Question: {question}\n")
print(f"The answer is choice {answer}: {options[answer]}")

======= Test Case 1 =======
Question: question                                     Searches a solution for
type                                                    MULTI_SELECT
options            [Scan business cards, Clean up CRM, Extract da...
intended_answer    [Extract data from emails, Improve CRM data qu...
context            I'm trying to find a way to extract data from ...
difficulty                                                      easy
label                                                [0, 0, 1, 1, 1]
stratify_key                                       easy_MULTI_SELECT
Name: 76, dtype: object

The answer is choice 4: Capture trade fair contacts


In [159]:
print(question_text)
print(context)
mc_results, results = model_output(model, tokenizer, tryout)

Searches a solution for
I'm trying to find a way to extract data from emails, improve the quality of my CRM data, and capture contacts I collect at trade fairs.
['Extract data from emails', 'Improve CRM data quality', 'Capture trade fair contacts']
['Clean up CRM', 'Improve CRM data quality', 'Capture trade fair contacts']


In [151]:
mc_results

[]